# 과제 1: 트랜잭션 실습해보기

## 트랜잭션이란?
### 동시에 Atomic 하게 실행되어야 하는 SQL들을 묶어서 하나의 작업처럼 처리하는 방법

In [5]:
import psycopg2

# Redshift connection 함수
def get_Redshift_connection(autocommit):
  host = "learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com"
  user = "jihoju96"
  password = "Jihoju96!1"
  port = 5439
  dbname = "dev"
  conn = psycopg2.connect(f"dbname={dbname} user={user} host={host} password={password} port={port}")
  conn.set_session(autocommit=autocommit)
  return conn

## INSERT SQL 을 autocommit=False 실행

- autocommit = False: 데이터를 insert or delete 해도 임시 테이블에 들어간다
  - 본인한테는 임시 테이블이 보이지만, 남한테는 안보인다.

In [6]:
conn = get_Redshift_connection(False)
cur = conn.cursor()

In [7]:
cur.execute("SELECT * FROM jihoju96.name_gender LIMIT 10;")
res = cur.fetchall()
for r in res:
  print(r)

('Adaleigh', 'F')
('Amryn', 'Unisex')
('Apurva', 'Unisex')
('Aryion', 'M')
('Alixia', 'F')
('Alyssarose', 'F')
('Arvell', 'M')
('Aibel', 'M')
('Atiyyah', 'F')
('Adlie', 'F')


In [8]:
cur.execute("DELETE FROM jihoju96.name_gender;")

In [10]:
cur.execute("INSERT INTO jihoju96.name_gender VALUES ('Jiho', 'Male')");

In [13]:
cur.execute("SELECT * FROM jihoju96.name_gender LIMIT 10;")
res = cur.fetchall()
for r in res:
  print(r)

('Jiho', 'Male')


- 임시테이블을 원본 테이블에 영구적으로 반영하고 싶다면 COMMIT
  - cur.execute("END;")
  - cur.execute("COMMIT;")
  - conn.commit()
- 임시 테이블을 처음으로(원본 테이블) 되돌리고 싶다면?
  - cur.execute("ROLLBACK;")
  - conn.rollback()

In [12]:
cur.execute("COMMIT;")

## INSERT SQL 을 autocommit=True로 실행하고 try/except로 컨트롤하기

In [14]:
conn = get_Redshift_connection(True)
cur = conn.cursor()

In [15]:
# 남한테도 delete 수행된 결과가 보인다. -> autocommit = True/False 차이점
cur.execute("DELETE FROM jihoju96.name_gender;")

In [16]:
try:
  cur.execute("BEGIN;")
  cur.execute("DELETE FROM jihoju96.name_gender;")
  cur.execute("INSERT INTO jihoju96.name_gender VALUES ('Claier', 'Female');")
  cur.execute("END;")
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
  cur.execute("ROLLBACK;")  # 임시로 작성되는 내용들을 버린다. -> 내 눈에도 이제 보이지 않는다.
finally:
  conn.close()

## INSERT SQL을 autocommit=True로 실행하고 SQL로 컨트롤하기

In [17]:
conn = get_Redshift_connection(True)
cur = conn.cursor()

In [20]:
cur.execute("SELECT * FROM jihoju96.name_gender;")
res = cur.fetchall()
for r in res:
  print(r)

('Benjamin', 'Male')


In [19]:
cur.execute("BEGIN;")
cur.execute("DELETE FROM jihoju96.name_gender;")
cur.execute("INSERT INTO jihoju96.name_gender VALUES ('Benjamin', 'Male');")

In [21]:
cur.execute("END;") # 임시 테이블에 쓰여진 내용이 원래 테이블에 영구적을 반영

## 잘못된 SQL을 중간에 실행하기

In [22]:
cur.execute("BEGIN;")
cur.execute("DELETE FROM jihoju96.name_gender;")
cur.execute("INSERT INTO jihoju96.name_gender3 VALUES ('Andrew', 'Male');")
cur.execute("END;")

UndefinedTable: ignored

=> 이런 의도치 않은 상황이 발생할 수 있으므로 try/except 문을 활용해서 컨트롤을 해주는 것이 안정적인 거 같다.